
# Modelos de Datos en Gestión de Big Data

In [ ]:

# === Setup para demos ===
import sqlite3, json, math, itertools
from collections import defaultdict, Counter
from datetime import datetime
try:
    import matplotlib.pyplot as plt  # Sólo se usa en una demo opcional de barras
except Exception as e:
    plt = None
print("Entorno listo. SQLite versión:", sqlite3.sqlite_version)



## **¿Por qué importa el modelo de datos?**

* Los **modelos de datos** son decisivos: influyen en cómo **escribimos** el software y en cómo **pensamos** el problema.
* Las aplicaciones **apilan** modelos por capas. En esencia:

  1. Del **mundo real** al **modelo de dominio** (estructuras y APIs propias de la app).
  2. De esas estructuras a un **modelo general** para persistir y consultar (JSON/XML, tablas relacionales, grafos).
  3. El **motor de base de datos** representa ese modelo en **bytes** (memoria, disco, red) para permitir consultas y procesamiento.
  4. El **hardware** representa los bytes como señales físicas.

  > Pueden existir APIs intermedias; cada capa **oculta complejidad** y permite que distintos equipos colaboren eficazmente.
* Todo modelo trae **supuestos y trade-offs**: algunas operaciones son naturales y rápidas; otras, difíciles o lentas.
* Elegir bien el modelo es crucial porque **limita o habilita** lo que el software puede hacer.
* Este notebook compara los modelos **relacional**, **documental** y **de grafo**, junto con sus **lenguajes de consulta** y casos de uso.



## **Modelo relacional vs. modelo documental**

* El **modelo relacional** (Codd, 1970) organiza los datos en **relaciones/tablas**; cada relación es una colección no ordenada de **tuplas/filas**.
* A mediados de los **años 80**, los **RDBMS** (relational database management systems) y **SQL** se consolidaron como la opción principal para almacenar y consultar datos con estructura regular; su **dominancia** se mantuvo por \~**25–30 años**.
* Origen: **procesamiento de datos de negocio** (años 60–70), con cargas típicas de **transacciones** (ventas, banca, reservas, inventario) y **batch** (facturación, nómina, reportes).
* Objetivo clave del modelo relacional: **ocultar detalles de implementación** interna detrás de una **interfaz limpia**, a diferencia de bases anteriores que obligaban al desarrollador a pensar en la representación física.
* Con el aumento del poder de cómputo y las redes, el relacional **generalizó bien** más allá del negocio: gran parte de la **web** (publicación, foros, redes sociales, e-commerce, juegos, SaaS) sigue respaldada por bases **relacionales**.

### **Surgimiemto de NoSQL**

* **NoSQL** surge en los **2010s** como intento de cuestionar la dominancia relacional. El término nació como un **hashtag** (2009) para un meetup sobre bases **no relacionales**, **distribuidas** y **open-source**; luego se reinterpretó como **“Not Only SQL”**.
* Bajo el paraguas NoSQL aparecen varios sistemas interesantes, sin referirse a una tecnología única.

**Impulsores clave de NoSQL:**

* Necesidad de **escalar** (difícil para RDBMS): **datos masivos** o **altísima tasa de escritura**.
* Preferencia por **software libre/open-source** frente a productos comerciales.
* **Consultas especializadas** poco naturales en el modelo relacional.
* **Esquemas rígidos** del mundo relacional → deseo de modelos más **dinámicos**.

Los requimiento actuales varían entre aplicaciones; esto conyeva a la necesidad de una **coexistencia** entre bases relacionales y diversas **no relacionales**.

### Desajuste entre objetos y relaciones

* La mayoría del desarrollo se hace en **lenguajes orientados a objetos**, mientras que los datos suelen almacenarse en **tablas relacionales**. Esto exige una **capa de traducción** entre *objetos* y *tablas/filas/columnas*, problema conocido como **impedance mismatch**.
* El mapeo objeto-relacional **ORM** reduce la cantidad de codigo adicional para esta traducción, pero **no elimina** las diferencias entre ambos modelos.

**Ejemplo (perfil tipo LinkedIn):**

* El perfil tiene un `user_id` y campos de ocurrencia única (`first_name`, `last_name`) que van como **columnas** en `users`.
* Otros datos son **uno-a-muchos** (empleos, educación, contactos).

<p align="center">
<image src="perfil_BGates.png" alt="Descripción de la imagen">
</p>

**Formas de representarlo en SQL:**

1. **Modelo clásico** (pre-SQL:1999): se usan tablas separadas para `positions`, `education`, `contact_info` con una **clave foránea** a la tabla `users`.
2. Estándares posteriores permiten datos **multivaluados** en una sola fila (**XML/JSON**), con cierto soporte de **consulta/índices** dentro de estos documentos (según motor).
3. Guardar **JSON/XML como texto** en una columna y dejar que la **aplicación** interprete su contenido (suele impedir consultar valores internos desde la BD).

<p align="center">
<image src="perfil_BG_SQL.png" alt="Descripción de la imagen">
</p>


**Por qué muchos usan JSON:**

* **Menos desajuste** con el código de objetos y mejor **localidad**: todo el perfil está en **un documento** (una sola lectura).
* En el esquema multi-tabla, recuperar el perfil requiere **múltiples consultas** o **joins** complejos.
* Esta sportado por varias bases de datos MongoDB, Espresso y otras

**Matiz importante:** el JSON hace explícita la **estructura en árbol** (uno-a-muchos), pero también tiene **limitaciones** como formato de almacenamiento; la “ausencia de esquema” puede ser ventaja o riesgo según el caso.

<p align="center">
<image src="tree_perfiles.png" alt="Descripción de la imagen">
</p>

### Documento (JSON) para un perfil simplificado
A continuación un perfil modelado como **documento**. Observa cómo agrupa posiciones y educación (estructura tipo **árbol**).

In [ ]:
{
    "user_id": 251,
    "user_id": 251,
    "first_name": "Bill",
    "last_name": "Gates",
    "summary": "Co-chair of the Bill & Melinda Gates... Active blogger.",
    "region_id": "us:91",
    "industry_id": 131,
    "photo_url": "/p/7/000/253/05b/308dd6e.jpg",
    "positions": [
    {"job_title": "Co-chair", "organization": "Bill & Melinda Gates Foundation"},
    {"job_title": "Co-founder, Chairman", "organization": "Microsoft"}
    ],
    "education": [
    {"school_name": "Harvard University", "start": 1973, "end": 1975},
    {"school_name": "Lakeside School, Seattle", "start": null, "end": null}
    ],
    "contact_info": {
    "blog": "http://thegatesnotes.com",
    "twitter": "http://twitter.com/BillGates"
}
}


**Ventaja:** una única lectura trae el **agregado completo**.  
**Cuidado:** cuando partes del documento referencian **otras entidades** (empresas, escuelas, otros usuarios), surge la necesidad de **referencias** y *joins*.


### Relaciones muchos-a-uno y muchos-a-muchos

* En el archivo .json anterior, `region_id` e `industry_id` aparecen como **IDs** y no como strings (“Greater Seattle Area”, “Philanthropy”).
  **Ventajas de usar IDs y listas estandarizadas** en lugar de texto libre:

  * Consistencia en estilo y ortografía.
  * Evitar ambigüedad (ciudades con mismo nombre).
  * Actualización sencilla (un solo lugar).
  * Localización en distintos idiomas.
  * Mejor búsqueda (p. ej., saber que *Seattle* está en *Washington*).

* Guardar texto en cada registro implica **duplicación**: si el nombre cambia, hay que actualizar todos los registros → riesgo de inconsistencias.
  Usar IDs evita esto: el ID permanece aunque el texto asociado cambie. Esta es la base de la **normalización** en bases de datos.

* La normalización genera naturalmente relaciones **muchos-a-uno** (muchos usuarios ↔ una región/industria).

  * En **relacional** es normal porque los *joins* son sencillos.
  * En **documentos**, un árbol uno-a-muchos funciona bien, pero los *joins* suelen ser limitados. Si no hay soporte nativo, deben emularse en la aplicación (con múltiples consultas o listas en memoria).

* Con el tiempo, los datos tienden a volverse más **interconectados**:

  * **Organizaciones y escuelas como entidades**: ya no simples cadenas, sino nodos con su propia información (logo, página, etc.). Cada perfil apunta a esas entidades
  * **Recomendaciones**: un usuario recomienda a otro → relación que conecta dos perfiles; si el autor cambia su foto, todas sus recomendaciones deben reflejarlo.

<p align="center">
  <image src="interconectados.png" alt="Descripción de la imagen">
</p>

* Estos casos producen **relaciones muchos-a-muchos**, que ya no encajan bien en un único documento. Se requieren **referencias** y *joins* al consultarlas.

<p align="center">
  <image src="muchosamuchos.png" alt="Descripción de la imagen">
</p>


### ¿Están repitiendo la historia las bases de documentos?

* Las **relaciones muchos-a-muchos** y los *joins* siempre fueron fáciles en **relacional**, pero en **documental/NoSQL** el debate resurge: ¿cómo representarlas mejor?
* Este debate es **previo a NoSQL**: ya existía en los **primeros sistemas de bases de datos**.

**Ejemplo histórico: IMS de IBM (1968).**

* Usaba el **modelo jerárquico**, muy parecido al de **documentos JSON**: todos los datos como un **árbol de registros anidados**.
* Funcionaba bien con relaciones **uno-a-muchos**, pero era problemático para **muchos-a-muchos** y no soportaba *joins*.
* Los desarrolladores debían **duplicar datos (desnormalizar)** o resolver referencias manualmente.
* Estos problemas de los **años 60–70** son casi los mismos que se encuentran hoy en bases documentales.

**Respuestas de la época:**

* **Modelo relacional** → evolucionó en SQL y se volvió dominante.
* **Modelo en red** → tuvo seguidores, pero perdió vigencia.
* El **“gran debate”** entre ambos ocupó gran parte de los 70.


### **El modelo en red**

* El **modelo en red** fue estandarizado por **CODASYL** (Conference on Data Systems Languages) y adoptado por varios proveedores. También se conoce como **modelo CODASYL**.
* Es una **generalización del modelo jerárquico**: en lugar de que cada registro tenga un único padre (árbol), un registro puede tener **múltiples padres**.

  * Ejemplo: un registro para *“Greater Seattle Area”* puede estar vinculado a todos los usuarios que viven allí → permite modelar relaciones **muchos-a-uno** y **muchos-a-muchos**.

**Claves del modelo:**

* Los enlaces entre registros no eran claves foráneas, sino **punteros** (guardados en disco).
* Para acceder a un registro había que seguir un **camino de acceso** desde un nodo raíz a través de esos enlaces.
* Una consulta consistía en mover un **cursor** por listas de registros y caminos de acceso.
* Si un registro tenía varios padres, el **código de aplicación** debía llevar la cuenta de todas esas relaciones.

**Problemas:**

* Los programadores tenían que recordar y gestionar múltiples **caminos de acceso**, lo que se describía como navegar en un **espacio de datos n-dimensional**.
* Esto era eficiente en el hardware limitado de los 70 (cintas magnéticas, accesos muy lentos), pero hacía el **código complejo e inflexible**.
* En jerárquico y en red, si no existía un camino predefinido hacia los datos requeridos, acceder a ellos resultaba muy difícil y obligaba a reescribir mucho código.


### **El modelo relacional**

* A diferencia de los modelos jerárquico y en red, el **modelo relacional** expone los datos de forma **simple y plana**:

  * Una **relación (tabla)** = colección de **tuplas (filas)**.
  * Sin estructuras anidadas ni caminos de acceso predefinidos.
  * Puedes leer cualquier fila filtrando por condiciones, o ubicar una fila específica usando una **clave**.
  * Insertar nuevas filas es directo, sin preocuparse por rutas de acceso complejas.

* En un RDBMS, el **optimizador de consultas** decide automáticamente:

  * El orden de ejecución de partes de la consulta.
  * Qué índices utilizar (*los índices son los caminos de acceso*).
    → El desarrollador no necesita programar manualmente esos caminos.

* Si quieres nuevas formas de consultar: basta con **crear un índice**. Las consultas ya existentes lo aprovechan sin cambios. Esto hizo mucho más fácil **extender aplicaciones**.

* Los **optimizadores relacionales** son sistemas complejos, fruto de décadas de investigación.

  * Idea clave: se construye **un solo optimizador** en el motor, y **todas las aplicaciones** se benefician.
  * Aunque escribir accesos manuales puede ser más simple al inicio, la solución general del optimizador **gana en el largo plazo**.

### **Comparación con bases documentales**

* Las **bases con modelo documental** retomaron del **modelo jerárquico** la idea de guardar **registros anidados** (uno-a-muchos) dentro de su registro padre (ej.: `positions`, `education`, `contact_info`) en lugar de usar tablas separadas.

* Para **muchos-a-uno** y **muchos-a-muchos**, sin embargo, **no difieren mucho del relacional**:

  * Se usan **identificadores únicos** para referenciar entidades relacionadas.
  * En relacional se llaman **claves foráneas (foreign keys)**; en documental, **referencias**.
  * En ambos casos, la resolución ocurre en la **lectura**, mediante un *join* o consultas adicionales.

* Hasta hoy, las bases documentales **no siguieron el camino del modelo CODASYL**, que exigía rutas de acceso manuales.



### **Relacional vs. Documental hoy en dia**

* Las diferencias clave entre modelos **no son solo técnicas internas** (tolerancia a fallos, concurrencia), sino de **cómo representan los datos**.

**Ventajas del modelo de documentos:**

* **Flexibilidad de esquema**.
* **Localidad**: mejor rendimiento cuando se consulta todo un agregado.
* Más cercano a las **estructuras de datos de la aplicación** (árboles uno-a-muchos).

**Ventajas del modelo relacional:**

* Mejor soporte para **joins**, **muchos-a-uno** y **muchos-a-muchos**.
* Esquemas más claros cuando hay **interconexión** fuerte entre datos.

**Simplicidad del código según el caso:**

* Si los datos son tipo **documento** (estructura árbol cargada completa), el modelo documental simplifica el código.
* El “**shredding**” (dividir un documento en varias tablas) en relacional puede generar esquemas engorrosos y lógica extra.
* Limitación: en documentos no puedes apuntar directo a un campo anidado; debes recorrer listas (similar a un acceso jerárquico).

**Relaciones muchos-a-muchos:**

* En analítica, quizá no importen → documentos funcionan bien.
* Si son necesarias, el modelo documental se complica:

  * Puedes **desnormalizar**, pero el código debe mantener consistencia.
  * O emular *joins* con varias consultas → más lento y más complejo.

**Conclusión:**

* No hay un ganador absoluto.
* **Documentos**: simples para datos en árbol con baja interconexión.
* **Relacional**: adecuado cuando hay relaciones complejas.
* **Grafos**: el modelo más natural para datos **altamente conectados**.


### **Flexibilidad de esquema en el modelo documental**

* En la mayoría de bases documentales (y JSON en RDBMS) **no se aplica un esquema obligatorio**: se pueden añadir claves/valores arbitrarios.
* Por eso se habla de bases **“schemaless”**, aunque en realidad siempre existe un **esquema implícito** asumido por el código que lee los datos.
* Términos más precisos:

  * **Schema-on-read**: la estructura solo se interpreta al leer (similar a **tipado dinámico** en lenguajes de programación).
  * **Schema-on-write**: el esquema es explícito y validado al escribir (similar a **tipado estático**).

**Cambio de formato de datos (ejemplo):**

* Antes: un campo `name` con el nombre completo.
* Ahora: separar en `first_name` y `last_name`.

  * En **documental**: basta con empezar a escribir con los nuevos campos y poner lógica en la aplicación para manejar documentos viejos.
  * En **relacional**: requiere **migración** (`ALTER TABLE` + `UPDATE`). Esto puede ser rápido (milisegundos en muchos motores) salvo en MySQL, que copia la tabla entera.

**Ventajas del enfoque schema-on-read:**

* Maneja mejor datos **heterogéneos** (colecciones con estructuras distintas).
* Útil cuando los datos vienen de sistemas externos que pueden cambiar.

**Ventajas del schema-on-write:**

* Documenta y garantiza que todos los registros tengan la misma estructura.
* Conveniente cuando los datos son **homogéneos**.

**no hay una respuesta única**; la elección depende de la **variabilidad de los datos** y de las **necesidades de control** sobre su estructura.


### Convergencia entre bases relacionales y documentales (síntesis traducida)

* Desde mediados de los 2000, la mayoría de **RDBMS** (excepto MySQL en sus primeras versiones) incorporaron soporte para **XML**:

  * Permite **modificaciones locales**, **índices** y **consultas internas** sobre documentos.

* Más tarde llegó el soporte nativo para **JSON**:

  * **PostgreSQL** (v9.3), **MySQL** (v5.7) y **IBM DB2** (v10.5) permiten consultas y funciones sobre JSON, lo que acerca sus capacidades al modelo documental.
  * La adopción de JSON en APIs web impulsó esta tendencia.

* En el lado documental:

  * **RethinkDB** incluye *joins* en su lenguaje.
  * **MongoDB** puede resolver referencias en drivers (equivalente a un *join* del lado del cliente, aunque menos eficiente que en el servidor).

**Tendencia:**

* Relacional y documentos se vuelven **cada vez más parecidos**, lo cual es positivo:

  * Los modelos **se complementan**.
  * Una base híbrida, que soporte tanto **datos tipo documento** como **consultas relacionales**, ofrece a las aplicaciones lo mejor de ambos mundos.

Cada vez vemos más **modelos híbridos** que combinen flexibilidad documental con la solidez de las consultas relacionales.


### **Lenguajes de consulta para datos**

* **Antes del modelo relacional** (IMS, CODASYL), las consultas eran con **código imperativo**: el programador definía paso a paso cómo recorrer y filtrar registros.
* Con el **modelo relacional**, surge **SQL**, un **lenguaje declarativo** inspirado en el álgebra relacional.

**Ejemplo imperativo (pseudocódigo):**

```js
function getSharks() {
  var sharks = [];
  for (var i=0; i<animals.length; i++) {
    if (animals[i].family === "Sharks") {
      sharks.push(animals[i]);
    }
  }
  return sharks;
}
```

**Álgebra relacional:**

$$
Sharks = σ_{family = Sharks}(animals)
$$

**SQL:**

```sql
SELECT * FROM animals WHERE family = 'Sharks';
```

**Diferencias clave:**

* **Imperativo:** especifica el *cómo* (orden de operaciones, bucles, variables).
* **Declarativo:** especifica el *qué* (condiciones, agrupaciones, ordenamientos), dejando al motor decidir cómo ejecutarlo.

**Ventajas del declarativo:**

* Más **conciso** y fácil de usar.
* Oculta detalles de implementación → el motor puede mejorar el rendimiento **sin cambiar las consultas**.
* No depende del **orden físico** de los datos, lo que permite reorganización interna sin romper resultados.
* Se presta mejor a la **ejecución paralela**: describe el resultado esperado, no el algoritmo.

**SQL y otros lenguajes declarativos** son más expresivos, portables y eficientes a largo plazo, porque liberan al programador de gestionar los detalles de acceso y optimización.

### **Consultas con MapReduce**

* Como vimos al inicio **MapReduce** es un modelo de programación para procesar **grandes volúmenes de datos** en paralelo en múltiples máquinas.
* En bases NoSQL como **MongoDB** o **CouchDB** se usa en forma limitada para consultas de solo lectura sobre muchos documentos.
* Está a medio camino entre **imperativo** y **declarativo**: el usuario escribe funciones (*map* y *reduce*) que el motor ejecuta repetidamente.

**Ejemplo (conteo de tiburones por mes):**

* En **SQL (PostgreSQL):**

```sql
SELECT date_trunc('month', observation_timestamp) AS obs_month,
       SUM(num_animals) AS total
FROM observations
WHERE family = 'Sharks'
GROUP BY obs_month;
```

* En **MongoDB con MapReduce:**

```js
db.observations.mapReduce(
  function map() {
    var y = this.observationTimestamp.getFullYear();
    var m = this.observationTimestamp.getMonth() + 1;
    emit(y + "-" + m, this.numAnimals);
  },
  function reduce(key, values) {
    return Array.sum(values);
  },
  {
    query: { family: "Sharks" },
    out: "monthlySharkReport"
  }
);
```

* **map**: emite `(año-mes, cantidad)` para cada documento.
* **reduce**: suma las cantidades agrupadas por clave.
* Ejemplo: dos observaciones en dic-1995 → `emit("1995-12",3)` y `emit("1995-12",4)` → `reduce("1995-12",[3,4]) = 7`.


**Restricciones y características:**

* Las funciones deben ser **puras** (sin efectos secundarios ni consultas extra).
* Esto permite reejecutarlas en cualquier orden, incluso tras fallos.
* Aunque son potentes (pueden parsear, calcular, usar librerías), escribir dos funciones coordinadas puede ser más complejo que una sola consulta SQL.


**Evolución en MongoDB:**

* **Agregation Pipeline** (desde v2.2): lenguaje declarativo con sintaxis JSON.

```js
db.observations.aggregate([
  { $match: { family: "Sharks" } },
  { $group: {
      _id: {
        year: { $year: "$observationTimestamp" },
        month: { $month: "$observationTimestamp" }
      },
      totalAnimals: { $sum: "$numAnimals" }
  } }
]);
```

* MapReduce es útil pero **bajo nivel**.
* Lenguajes declarativos (SQL o pipelines) son más concisos, optimizables y fáciles de usar.
* En la práctica, muchos sistemas NoSQL terminan **reinventando SQL** con otra sintaxis.


### **Modelos de datos tipo grafo**
* Cuando los datos son **uno-a-muchos** (estructuras de árbol) o sin relaciones, el **modelo documental** funciona bien.
* Pero si las **muchos-a-muchos** son frecuentes y complejas, resulta más natural usar un **grafo**.

**Definiciones básicas:**

* Un **grafo** tiene:

  * **Vértices (nodos/entidades).**
  * **Aristas (relaciones/enlaces).**

**Ejemplos típicos:**

* **Redes sociales:** vértices = personas, aristas = amistades.
* **Web graph:** vértices = páginas web, aristas = enlaces HTML.
* **Redes viales/ferrocarril:** vértices = cruces, aristas = carreteras o vías.

**Algoritmos comunes en grafos:**

* **Camino más corto** (navegación de rutas).
* **PageRank** (popularidad de páginas en la web).

**Grafos heterogéneos:**
No todos los vértices deben ser del mismo tipo. Ejemplo: en Facebook, el grafo incluye personas, ubicaciones, eventos, publicaciones y comentarios, todos interconectados.

**Ejemplo:**

* Personas: **Lucy** (de Idaho, EE. UU.) y **Alain** (de Beaune, Francia).
* Están **casados** y viven en **Londres**.
* El grafo integra diferentes tipos de nodos (personas, ciudades, países, continentes) y relaciones (`born_in`, `lives_in`, `within`, `married`).

<p align="center">
  <image src="graph_model.png" alt="Descripción de la imagen">
</p>

**Modelos y lenguajes para grafos:**

* **Property Graph** (Neo4j, Titan, InfiniteGraph).
* **Triple-store** (Datomic, AllegroGraph, etc.).
* Lenguajes **declarativos**: **Cypher**, **SPARQL**, **Datalog**.
* También existen enfoques **imperativos** (ej. **Gremlin**) y frameworks de **procesamiento** (ej. **Pregel**).

Los grafos permiten representar datos **altamente conectados** y consultarlos de forma más natural que los modelos relacional o documental.



## **¿Cuándo elegir qué? (Checklist rápida)**
- **Documento (JSON)** si predomina la lectura de **agregados completos** con jerarquía **uno‑a‑muchos**, y el esquema cambia con frecuencia.
- **Relacional** si necesitas **consultas ad‑hoc** complejas, **muchos‑a‑muchos** y **consistencia** fuerte.
- **Grafo** si las preguntas son sobre **caminos** de longitud variable (recomendación, redes, permisos).
- **Batch/MapReduce** si hay agregaciones masivas.
- Considera enfoques **híbridos** (p. ej., JSON en RDBMS; referencias y resoluciones en bases de documentos).


## **Ejercicio:**


### Crear bese de datos relacional pra el ejemplo de los perfiles en LinkedIn


In [ ]:

# Construimos un esquema relacional mínimo
con = sqlite3.connect(":memory:")
cur = con.cursor()

cur.executescript("""
DROP TABLE IF EXISTS users;
DROP TABLE IF EXISTS organizations;
DROP TABLE IF EXISTS positions;
DROP TABLE IF EXISTS schools;
DROP TABLE IF EXISTS education;

CREATE TABLE users (
  user_id INTEGER PRIMARY KEY,
  first_name TEXT,
  last_name  TEXT,
  summary    TEXT,
  location   TEXT
);

CREATE TABLE organizations (
  org_id INTEGER PRIMARY KEY,
  name   TEXT UNIQUE
);

CREATE TABLE positions (
  pos_id INTEGER PRIMARY KEY,
  user_id INTEGER,
  title   TEXT,
  org_id  INTEGER,
  start_year INTEGER,
  end_year   INTEGER,
  FOREIGN KEY (user_id) REFERENCES users(user_id),
  FOREIGN KEY (org_id)  REFERENCES organizations(org_id)
);

CREATE TABLE schools (
  school_id INTEGER PRIMARY KEY,
  name TEXT UNIQUE
);

CREATE TABLE education (
  edu_id INTEGER PRIMARY KEY,
  user_id INTEGER,
  school_id INTEGER,
  start_year INTEGER,
  end_year   INTEGER,
  FOREIGN KEY (user_id)  REFERENCES users(user_id),
  FOREIGN KEY (school_id) REFERENCES schools(school_id)
);
""")

# Insertamos datos
cur.execute("INSERT INTO users VALUES (?,?,?,?,?)",
            (251, "Bill", "Gates",
             "Co‑chair en B&M Gates Foundation; co‑fundador de Microsoft.", "Seattle, WA"))
orgs = ["Microsoft", "Bill & Melinda Gates Foundation"]
for o in orgs:
    cur.execute("INSERT INTO organizations(name) VALUES (?)", (o,))

schools = ["Harvard University", "Lakeside School"]
for s in schools:
    cur.execute("INSERT INTO schools(name) VALUES (?)", (s,))

# Ayuda utilitaria para obtener id
def get_id(table, name_field, name_value, id_field):
    cur.execute(f"SELECT {id_field} FROM {table} WHERE {name_field}=?", (name_value,))
    return cur.fetchone()[0]

cur.execute("INSERT INTO positions(user_id,title,org_id,start_year,end_year) VALUES (?,?,?,?,?)",
            (251, "Co‑founder", get_id("organizations","name","Microsoft","org_id"), None, None))
cur.execute("INSERT INTO positions(user_id,title,org_id,start_year,end_year) VALUES (?,?,?,?,?)",
            (251, "Co‑chair", get_id("organizations","name","Bill & Melinda Gates Foundation","org_id"), None, None))

cur.execute("INSERT INTO education(user_id,school_id,start_year,end_year) VALUES (?,?,?,?)",
            (251, get_id("schools","name","Harvard University","school_id"), 1973, 1975))
cur.execute("INSERT INTO education(user_id,school_id,start_year,end_year) VALUES (?,?,?,?)",
            (251, get_id("schools","name","Lakeside School","school_id"), None, None))

con.commit()

# Consulta para reconstruir el "agregado perfil"
query_positions = '''
SELECT u.user_id, u.first_name || ' ' || u.last_name AS name,
       p.title, o.name AS organization
FROM users u
JOIN positions p ON p.user_id = u.user_id
JOIN organizations o ON o.org_id = p.org_id
WHERE u.user_id = 251
ORDER BY p.pos_id;
'''
query_education = '''
SELECT u.user_id, s.name AS school, e.start_year, e.end_year
FROM users u
JOIN education e ON e.user_id = u.user_id
JOIN schools s   ON s.school_id = e.school_id
WHERE u.user_id = 251
ORDER BY e.edu_id;
'''
print("-- POSICIONES --")
for row in cur.execute(query_positions):
    print(row)
print("\n-- EDUCACIÓN --")
for row in cur.execute(query_education):
    print(row)
